In [1]:
import numpy as np
import scipy as sp

In [25]:
data_path = "../Data/iPad_s8/iPad_s8_train_data_3.txt"

label_arr = []
feature_arr = []

with open(data_path, 'r') as data_file:
        for line in data_file:
            value = line.split('|')
            
            label_line = value[1]
            label_rgb = label_line.split(' ')
            label_arr.append([int(label_rgb[1]), int(label_rgb[2]), int(label_rgb[3])])
            
            feature_line = value[2]
            feature_rgb = feature_line.split(' ')
            feature_arr.append([1, int(feature_rgb[1]), int(feature_rgb[2]), int(feature_rgb[3].replace('\n', ''))])
            
#print(label_arr[:10])
print(len(label_arr))

#print(feature_arr[:10])
print(len(feature_arr))

11847024
11847024


In [26]:
x = np.array(feature_arr) # target value => S8
#print("x : ", x[:10])
xt = x.transpose()
#print("xt : ", xt[:10])

y = np.array(label_arr) # original value => iPad Mini 4
#print("y : ", y[:10])

In [27]:
# Find w = (XtX)^-1 Xt Y
prefix = np.matmul(xt, x)
prefix_inv = np.linalg.pinv(prefix)
w = np.matmul(prefix_inv, np.matmul(xt, y))
print(w)

[[ -4.19177110e+00  -1.88891061e+00  -2.94830842e-02]
 [  9.09799628e-01   5.31777804e-02  -1.35308044e-02]
 [ -5.29412699e-02   8.65753128e-01   1.88806811e-02]
 [ -1.14148538e-03  -5.45939643e-02   7.91649258e-01]]


In [28]:
np.matmul(x, w)

array([[  30.40484047,   91.21372668,  142.46535772],
       [  48.62137087,  142.96221838,   75.86463836],
       [ 108.0045137 ,  134.4077601 ,  161.14274047],
       ..., 
       [  42.8812287 ,   82.17112787,  132.56309799],
       [  43.85875762,  142.67307857,  113.98809883],
       [ 124.27045853,  136.87809054,  164.10354439]])

In [41]:
from PIL import Image

image_path = "./example_image/Arm(S8+)_resize.jpg"
img = Image.open(image_path)
pixel_list = []
for i in range(img.size[0]):    
        for j in range(img.size[1]):
            pixel_list.append(img.getpixel((i,j)))
            #print("R : ", img.getpixel((i,j))[0])

#print(pixel_list[:10])
#print(len(pixel_list))

pixel_set = set(pixel_list)
original_value = list(pixel_set)
#print(original_value[:10])
#print(len(original_value))

original_unique_value = []
for point in original_value:
    original_unique_value.append([1, point[0], point[1], point[2]])

original_value_arr = np.array(original_unique_value)
#print(original_value_arr[:10])


In [42]:
predict_arr = np.matmul(original_value_arr, w)
print(predict_arr)
print(len(predict_arr))

[[  75.01930921   84.55469707   74.22360125]
 [  90.54336771   95.71390185  114.63736917]
 [ 181.11502136  165.89515548  131.28973171]
 ..., 
 [  48.43615336   55.08514756   88.30744172]
 [ 139.55481731  103.1481441    85.46397448]
 [ 139.61703891  142.23556698  137.01408797]]
57721


In [43]:
combined_dict = {}
i = 0
for e in original_value:
    combined_dict[e] = predict_arr[i]
    i += 1

i = 0
for key in combined_dict:
    print(key , " => ", combined_dict[key])
    i+=1
    if i > 5:
        break

(93, 100, 93)  =>  [ 75.01930921  84.55469707  74.22360125]
(111, 115, 144)  =>  [  90.54336771   95.71390185  114.63736917]
(215, 191, 165)  =>  [ 181.11502136  165.89515548  131.28973171]
(216, 217, 211)  =>  [ 180.59583964  185.94659222  168.1829645 ]
(32, 28, 42)  =>  [ 23.39151906  21.76091944  33.3154591 ]
(73, 42, 40)  =>  [ 59.95440902  36.17094016  31.44172714]


In [44]:
def correct_range(pixel_value):
    if pixel_value < 0:
        pixel_value = 0
        
    if pixel_value > 255:
        pixel_value = 255
        
    return pixel_value

img = Image.open(image_path)
pixels = img.load()
for i in range(img.size[0]):    
        for j in range(img.size[1]):
            key = img.getpixel((i,j))
            R = int(combined_dict[key][0])
            G = int(combined_dict[key][1])
            B = int(combined_dict[key][2])
            
            R = correct_range(R)
            G = correct_range(G)
            B = correct_range(B)
            
            pixels[i,j] = (R, G, B)
            
img.save("./example_image/iPad_s8_augmented_img/Arm(S8+)_resize_Nannnnn.jpg")
print("Finish Write Image")

Finish Write Image


In [59]:
from IPython.display import HTML, display
feature_img = "./example_image/Arm(S8+)_resize.jpg"
label_img = "./example_image/Arm(iPad)_resize.jpg"
nan_img = "./example_image/iPad_s8_augmented_img/Arm(S8+)_resize_Nannnnn.jpg"
my_own_img = "./example_image/iPad_s8_augmented_img/Arm(S8+)_resize_Edited_By_exp-27_model-15(Avg).jpg"

display(HTML("<table><tr><td>Galaxy S8+<img src='"+ feature_img +"'></td><td>P' Nan<img src='"+ nan_img +"'></td><td>iPad Mini 4<img src='"+ label_img +"'></td></tr></table>"))
display(HTML("<table><tr><td>Galaxy S8+<img src='"+ feature_img +"'></td><td>Mine<img src='"+ my_own_img +"'></td><td>iPad Mini 4<img src='"+ label_img +"'></td></tr></table>"))

Galaxy S8+,P' Nan,iPad Mini 4


Galaxy S8+,Mine,iPad Mini 4


In [60]:
feature_img = "./example_image/TheTwo(S8+)_resize.jpg"
label_img = "./example_image/TheTwo(iPad)_resize.jpg"
nan_img = "./example_image/iPad_s8_augmented_img/TheTwo(S8+)_resize_Nannnnn.jpg"
my_own_img = "./example_image/iPad_s8_augmented_img/TheTwo(S8+)_resize_Edited_By_exp-27_model-15(Avg).jpg"

display(HTML("<table><tr><td>Galaxy S8+<img src='"+ feature_img +"'></td><td>P' Nan<img src='"+ nan_img +"'></td><td>iPad Mini 4<img src='"+ label_img +"'></td></tr></table>"))
display(HTML("<table><tr><td>Galaxy S8+<img src='"+ feature_img +"'></td><td>Mine<img src='"+ my_own_img +"'></td><td>iPad Mini 4<img src='"+ label_img +"'></td></tr></table>"))

Galaxy S8+,P' Nan,iPad Mini 4


Galaxy S8+,Mine,iPad Mini 4
